In [0]:
# variables
storage_account_name = "voc43cdmqlsgj6nostorage"
container_name = "data"
mount_point = "/mnt/data"

# Securely fetch the secret key from the scope we created
try:
    secret_key = dbutils.secrets.get(scope="voc_project_scope", key="storage-account-key")
except Exception as e:
    print("Looks like your secret scope or key isn't set up.")
    raise e

# Build the configuration string
config = f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net"

# Check if the mount point already exists (makes our code safe to re-run)
if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    print(f"Mounting {mount_point}...")
    
    # Mount the storage
    dbutils.fs.mount(
      source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
      mount_point = mount_point,
      extra_configs = {config: secret_key}
    )
    print(f"Successfully mounted {mount_point}!")
else:
    print(f"{mount_point} is already mounted.")

# Verify by listing the files
print("--- Files in mount point ---")
display(dbutils.fs.ls(mount_point))